In [ ]:
# pip install pycryptodome

In [ ]:
# pip install opencv-python

In [1]:
#importing the necessary libraries
import os
import sys
import cv2
import numpy as np
import shutil

In [2]:
def load_frames(mode = "train"):
    """
    Extract frames from the video and save them as JPEG images in a folder
    Args:
        mode (str): either "train" or "test", determines which video to load
    Returns:
        numpy.ndarray: an array containing all of the frames from the video
    """
    # Checking if directory exists, if it does remove it
    if os.path.isdir("./data"):
        shutil.rmtree("./data")

    # Creating a directory to store frames
    dir_path = os.path.join("./data", mode+'_imgs')
    os.makedirs(dir_path, exist_ok=True)

    # Opening the video file
    video_path = 'assets/train.mp4'
    vidcap = cv2.VideoCapture(video_path)

    # Extracting frames from the video and save as JPEG files
    success,image = vidcap.read()
    count = 0
    while success:
        cv2.imwrite("%s/%d.jpg" % (dir_path,count), image)     # save frame as JPEG file      
        success,image = vidcap.read()
        count += 1
        if count % 300 == 0:
            print("%d frames read" % count)

    # Loading the saved JPEG files into a numpy array
    frame_cnt = len(os.listdir(dir_path))
    vid_frames = np.empty((frame_cnt,1080,1920,3),dtype='uint8')
    for i in range(0,frame_cnt):
        frame = cv2.imread(dir_path + '/' + str(i) + ".jpg")
        vid_frames[i] = frame
        sys.stdout.write("\rLoading frame " + str(i))

    return vid_frames

# Calling the function to extract frames from the "train" video
load_frames()


300 frames read
600 frames read
Loading frame 744

array([[[[138, 102,  54],
         [138, 102,  54],
         [138, 102,  54],
         ...,
         [136,  97,  65],
         [136,  97,  65],
         [136,  97,  65]],

        [[138, 102,  54],
         [138, 102,  54],
         [138, 102,  54],
         ...,
         [136,  97,  65],
         [136,  97,  65],
         [136,  97,  65]],

        [[138, 102,  54],
         [138, 102,  54],
         [138, 102,  54],
         ...,
         [136,  97,  65],
         [136,  97,  65],
         [136,  97,  65]],

        ...,

        [[  1,   1,   1],
         [  1,   1,   1],
         [  1,   1,   1],
         ...,
         [  4,   0,   0],
         [  4,   0,   0],
         [  4,   0,   0]],

        [[  1,   1,   1],
         [  1,   1,   1],
         [  1,   1,   1],
         ...,
         [  4,   0,   0],
         [  4,   0,   0],
         [  4,   0,   0]],

        [[  1,   1,   1],
         [  1,   1,   1],
         [  1,   1,   1],
         ...,
         [  4,   0,   0],
        

In [9]:
import numpy as np
import time

video_path = 'assets/train.mp4'
cap = cv2.VideoCapture(video_path)

# Initialize the previous frame and previous points for optical flow
prev_frame = None
prev_pts = None

# Initialize a font for displaying the speed value
font = cv2.FONT_HERSHEY_SIMPLEX

# Set the reference distance in meters and the reference time in seconds
ref_distance = 10  # 10 meters
ref_time = 3  # 3 seconds

# Initialize an empty list to store the speed values
speeds = []

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # If this is not the first frame, compute optical flow
        if prev_frame is not None:
            # Compute optical flow using the previous frame and the current frame
            flow = cv2.calcOpticalFlowFarneback(prev_frame, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            
            # Compute the magnitude and angle of the optical flow vectors
            mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
            
            # Convert the angle to hue value
            hsv = np.zeros_like(frame)
            hsv[...,0] = ang * 180 / np.pi / 2
            
            # Set the value and saturation to maximum
            hsv[...,1] = 255
            hsv[...,2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
            
            # Convert the HSV image to BGR for display
            bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
            
            # Overlay the optical flow vectors on top of the frame
            frame = cv2.addWeighted(frame, 1, bgr, 2, 0)
            
            # Compute the speed of the car based on the mean magnitude of optical flow vectors
            speed = np.mean(mag) * ref_distance / ref_time
            
            # Convert the speed value from meters per second to kilometers per hour
            speed_kmph = speed * 3.6
            
            # Draw the speed value on top of the frame
            cv2.putText(frame, 'Speed: {:.2f} km/h'.format(speed_kmph), (10, 50), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
            
            # Display the frame with optical flow vectors and speed value
            cv2.imshow('frame', frame)
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
            
            # Append the speed value to the list of speeds
            speeds.append(speed_kmph)
        
        # Update the previous frame and previous points for optical flow
        prev_frame = gray.copy()
        prev_pts = cv2.goodFeaturesToTrack(prev_frame, 200, 0.01, 10)
    else:
        break

cap.release()
cv2.destroyAllWindows()

# Convert the list of speeds to a NumPy array
speeds = np.array(speeds)

# Print the mean and standard deviation of the speeds
print('Mean speed: {:.2f} km/h'.format(np.mean(speeds)))
print('Std dev of speed: {:.2f} km/h'.format(np.std(speeds)))

1   HIToolbox                           0x000000019e9a790c _ZN15MenuBarInstance22RemoveAutoShowObserverEv + 44
2   HIToolbox                           0x000000019e9e6478 _ZL17BroadcastInternaljPvh + 184
3   SkyLight                            0x0000000199ef9214 _ZN12_GLOBAL__N_123notify_datagram_handlerEj15CGSDatagramTypePvmS1_ + 896
4   SkyLight                            0x0000000199ef7e10 CGSSnarfAndDispatchDatagrams + 808
5   SkyLight                            0x000000019a21fa9c SLSGetNextEventRecordInternal + 344
6   SkyLight                            0x000000019a020fb0 SLEventCreateNextEvent + 16
7   HIToolbox                           0x000000019e9b4b58 _ZL38PullEventsFromWindowServerOnConnectionjhP17__CFMachPortBoost + 60
8   HIToolbox                           0x000000019e9b4ae0 _ZL14MessageHandlerP12__CFMachPortPvlS1_ + 60
9   CoreFoundation                      0x00000001952f9410 __CFMachPortPerform + 260
10  CoreFoundation                      0x00000001952c9f98 __CFRUNLO

Mean speed: 29.81 km/h
Std dev of speed: 5.35 km/h


1   HIToolbox                           0x000000019ea1c5c8 _ZN15MenuBarInstance22EnsureAutoShowObserverEv + 120
2   HIToolbox                           0x000000019e9e6478 _ZL17BroadcastInternaljPvh + 184
3   SkyLight                            0x0000000199ef9214 _ZN12_GLOBAL__N_123notify_datagram_handlerEj15CGSDatagramTypePvmS1_ + 896
4   SkyLight                            0x0000000199ef7e10 CGSSnarfAndDispatchDatagrams + 808
5   SkyLight                            0x000000019a21fa9c SLSGetNextEventRecordInternal + 344
6   SkyLight                            0x000000019a020fb0 SLEventCreateNextEvent + 16
7   HIToolbox                           0x000000019e9b4b58 _ZL38PullEventsFromWindowServerOnConnectionjhP17__CFMachPortBoost + 60
8   HIToolbox                           0x000000019e9b4ae0 _ZL14MessageHandlerP12__CFMachPortPvlS1_ + 60
9   CoreFoundation                      0x00000001952f9410 __CFMachPortPerform + 260
10  CoreFoundation                      0x00000001952c9f98 __CFRUNL

In [11]:
# Exporting the speeds array to a file
np.save('speeds.npy', speeds)

In [12]:
loaded_arr = np.load('speeds.npy')
print(loaded_arr)

[38.70598125 38.76813984 40.53661251 39.68199062 40.61893559 41.62860203
 38.49243164 39.54121685 36.1076889  39.49189854 38.56394577 44.7014122
 47.4123745  51.82500458 49.4827137  46.4105444  39.2718401  38.20156002
 35.82589245 29.65805054 30.01473999 33.6386776  33.37495136 35.75753117
 33.79273224 33.43532467 35.37848854 38.07975197 39.18135452 42.27786827
 43.19677162 35.98421288 36.0061512  31.4195509  32.55820656 33.19928455
 38.19158363 41.63331127 41.30067444 38.97743797 37.24567223 33.3965292
 34.14615726 36.64096928 37.49496174 35.06815052 35.83380032 37.42049217
 33.61771774 35.031847   35.03550911 35.0200367  36.84408474 36.38527679
 31.80350304 31.21949387 27.80244541 29.99814892 26.45937538 25.14543056
 27.10518837 26.08310795 27.99552441 29.21648026 32.11931992 35.01823997
 35.39827824 35.34638786 35.3577919  34.53490448 39.3079834  38.35517979
 38.76276112 36.91597652 35.65488052 35.52755928 37.65610886 37.17348289
 37.20189857 37.34794807 36.03048134 35.77456284 32.0

In [13]:
from Crypto.Cipher import AES
import os

def encrypt_file_AES(key, in_filename, out_filename=None, chunksize=64*1024):
    """Encrypts a file using AES encryption.

    Args:
        key (bytes): The encryption key. Must be 16, 24, or 32 bytes long.
        in_filename (str): The name of the input file.
        out_filename (str): The name of the output file. If not specified, 
            will be set to the input file name with '.enc' extension.
        chunksize (int): The size of each chunk in bytes.

    Returns:
        str: The name of the output file.
    """
    # Set the output file name if not specified
    if not out_filename:
        out_filename = in_filename + '.enc'

    # Generate a random IV
    iv = os.urandom(16)

    # Create an AES cipher object
    cipher = AES.new(key, AES.MODE_CBC, iv)

    # Open the input and output files
    with open(in_filename, 'rb') as infile, open(out_filename, 'wb') as outfile:
        # Write the IV to the output file
        outfile.write(iv)

        # Encrypt each chunk of the input file and write it to the output file
        while True:
            chunk = infile.read(chunksize)
            if len(chunk) == 0:
                break
            elif len(chunk) % 16 != 0:
                # Pad the chunk to a multiple of 16 bytes
                chunk += b' ' * (16 - len(chunk) % 16)
            encrypted_chunk = cipher.encrypt(chunk)
            outfile.write(encrypted_chunk)

    return out_filename


In [14]:
# Convert the key string to bytes and truncate it to 16 bytes
key = 'this_is_a_16_byte_key'.encode('utf-8')[:16]

# Encrypt the 'speeds.npy' file using AES encryption
encrypted_file = encrypt_file_AES(key, 'speeds.npy')


In [15]:
from Crypto.Cipher import AES

def decrypt_file_AES(key, in_filename, out_filename=None, chunksize=24*1024):
    """Decrypts a file that has been encrypted using AES encryption.

    Args:
        key (bytes): The encryption key. Must be 16, 24, or 32 bytes long.
        in_filename (str): The name of the input file.
        out_filename (str): The name of the output file. If not specified, 
            will be set to the input file name with '.dec' extension.
        chunksize (int): The size of each chunk in bytes.

    Returns:
        str: The name of the output file.
    """
    # Set the output file name if not specified
    if not out_filename:
        out_filename = in_filename + '.dec'

    # Open the input and output files
    with open(in_filename, 'rb') as infile, open(out_filename, 'wb') as outfile:
        # Read the IV from the input file
        iv = infile.read(16)

        # Create an AES cipher object
        cipher = AES.new(key, AES.MODE_CBC, iv)

        # Decrypt each chunk of the input file and write it to the output file
        while True:
            chunk = infile.read(chunksize)
            if len(chunk) == 0:
                break
            decrypted_chunk = cipher.decrypt(chunk)
            outfile.write(decrypted_chunk)

    return out_filename


In [22]:
# Load the key from a file or use the same key that was used for encryption
key = 'this_is_a_16_byte_key'.encode('utf-8')[:16]

# Decrypt the encrypted 'speeds.npy' file using AES decryption
decrypted_file = decrypt_file_AES(key, 'speeds.npy.enc')


In [24]:
# Loading the original 'speeds.npy' file
speeds_orig = np.load('speeds.npy')

# Loading the decrypted 'speeds.npy.enc.dec' file
speeds_dec = np.load('speeds.npy.enc.dec')

# Verifing that the arrays are equal
if np.array_equal(speeds_orig, speeds_dec):
    print("The arrays are equal. The decryption was completed successfully")
else:
    print("The arrays are not equal.")


The arrays are equal. The decryption was completed successfully
